In [1]:
import numpy as np

def objective(x):
    """目标函数：例如最小化 x[0]^2 + x[1]^2"""
    return x[0]**2 + x[1]**2

def constraint(x):
    """约束条件：例如 g(x) = x[0] + x[1] - 1，g(x) >= 0"""
    return x[0] + x[1] - 1

def linear_approximation(func, xk, perturbation=1e-5):
    """线性化函数，通过对目标函数和约束函数进行泰勒展开，估算其梯度"""
    n = len(xk)
    approx = np.zeros(n)
    for i in range(n):
        xk_perturbed = xk.copy()
        xk_perturbed[i] += perturbation
        approx[i] = (func(xk_perturbed) - func(xk)) / perturbation
    return approx

def cobyla(objective, constraint, x0, max_iter=100, tol=1e-6, rho=1.0):
    """COBYLA算法"""
    xk = np.array(x0)  # 当前点
    n = len(x0)
    m = 1  # 只有一个约束g(x) >= 0
    for iteration in range(max_iter):
        # 1. 计算目标和约束的线性化
        grad_f = linear_approximation(objective, xk)  # 目标函数的梯度
        grad_g = linear_approximation(constraint, xk)  # 约束函数的梯度
        f_xk = objective(xk)
        g_xk = constraint(xk)
        
        # 2. 构建单纯形
        simplex = np.array([xk + np.random.randn(n) * rho for _ in range(n+1)])
        
        # 3. 线性拟合：目标函数和约束的近似
        a0 = grad_f
        b0 = f_xk
        ag = grad_g
        bg = g_xk
        
        # 4. 解线性优化子问题
        # 在信赖域内最小化线性近似模型，目标是最小化 a0^T d
        # 线性约束为 ag^T d + bg >= 0, 并且 ||d|| <= rho
        d = np.linalg.solve(np.outer(a0, a0) + np.outer(ag, ag), -a0)
        
        # 5. 更新点和信赖域
        xk_next = xk + d
        if objective(xk_next) < f_xk:
            xk = xk_next
            rho *= 1.2  # 扩大信赖域
        else:
            rho *= 0.5  # 缩小信赖域
        
        # 6. 收敛判断
        if np.linalg.norm(d) < tol:
            print(f"Converged at iteration {iteration}")
            break

    return xk

# 初始点
x0 = [0.5, 0.5]
result = cobyla(objective, constraint, x0)
print(f"Optimization result: {result}")


LinAlgError: Singular matrix

In [55]:
import numpy as np

def cobyla(func, x0, constraints, max_iter=100, tol=1e-6, rho=0.01, sigma=0.1, trust_region=1.0):
    """
    COBYLA算法的自定义实现，用于求解带约束的非线性优化问题。
    
    参数：
    - func: 目标函数，接收参数x（向量）并返回目标函数值。
    - x0: 初始猜测点，必须是一个n维向量。
    - constraints: 约束条件的函数列表，每个约束函数都返回一个标量值。
    - max_iter: 最大迭代次数。
    - tol: 收敛容忍度（目标函数变化小于此值时认为已经收敛）。
    - rho: 初始步长系数。
    - sigma: 步长缩放因子。
    - trust_region: 初始信赖区域大小。
    
    返回：
    - 最优解和最优值。
    """
    
    n = len(x0)
    x = np.array(x0)  # 当前解
    f_x = func(x)  # 当前目标函数值
    constr_vals = np.array([constr(x) for constr in constraints])  # 当前约束值
    
    # 计算约束函数的最大值
    max_constr = np.max(constr_vals)
    
    # 初始化一些辅助变量
    iter_count = 0
    rho_factor = rho * np.linalg.norm(x0)
    trust_radius = trust_region  # 初始化信赖区域大小
    
    while iter_count < max_iter:
        # 计算当前解的梯度（使用有限差分法）
        grad_f = linear_approximation(func, x)
        
        # 计算约束函数的梯度
        grad_constraints = np.array([linear_approximation(constr, x) for constr in constraints])
        
        # 计算步长方向
        step_direction = -grad_f
        
        # 按照梯度下降更新解
        x_new = x + trust_radius * step_direction
        
        # 计算新的目标函数值和约束值
        f_x_new = func(x_new)
        constr_vals_new = np.array([constr(x_new) for constr in constraints])
        
        # 评估当前步长的效果
        if abs(f_x_new - f_x) < tol and np.all(constr_vals_new <= 0):
            print("Optimization converged!")
            return x_new, f_x_new
        
        # 如果新的解更好，扩大信赖区域
        if f_x_new < f_x and np.all(constr_vals_new <= 0):
            trust_radius *= 1.1  # 放大信赖区域
        else:
            trust_radius *= 0.5  # 缩小信赖区域

        # 更新当前解和目标函数值
        x = x_new
        f_x = f_x_new
        constr_vals = constr_vals_new
        iter_count += 1

    print("Reached max iterations.")
    return x, f_x

def linear_approximation(func, xk, perturbation=1e-5):
    """线性化函数，通过对目标函数和约束函数进行泰勒展开，估算其梯度"""
    n = len(xk)
    approx = np.zeros(n)
    for i in range(n):
        xk_perturbed = xk.copy()
        xk_perturbed[i] += perturbation
        approx[i] = (func(xk_perturbed) - func(xk)) / perturbation
    return approx

# 目标函数示例：简单的二次目标函数
def objective(x):
    return x[0]**2 + x[1]**2

# 约束函数示例：简单的线性约束
def constraint_1(x):
    return x[0] + x[1] - 1  # x1 + x2 <= 1

def constraint_2(x):
    return x[0] - 0.5  # x1 >= 0.5

# 初始猜测
x0 = [0.8, 0.3]

# 约束函数列表
constraints = [constraint_1, constraint_2]

# 调用COBYLA算法
solution, optimal_value = cobyla(objective, x0, constraints)

print("Optimal solution:", solution)
print("Optimal objective value:", optimal_value)


Optimization converged!
Optimal solution: [-4.99999988e-06 -4.99999965e-06]
Optimal objective value: 4.999999529708542e-11


In [53]:
from scipy.optimize import minimize
import nevergrad as ng
from nevergrad.functions import ArtificialFunction

D = 2
budget = 200
objective = ArtificialFunction(
        name="rastrigin",
        block_dimension=D,
        noise_level=False,
        rotation=False,
        translation_factor=10
    )



def constraint(x):
    """约束条件：例如 g(x) = x[0] + x[1] - 1，g(x) >= 0"""
    return x[0] + x[1] - 10

# 初始点
x0 = [-10.5, 0.5]

# 约束条件
con = {'type': 'ineq', 'fun': constraint}

result = minimize(objective, x0,  method='COBYLA')
best_value = objective(result.x)
print(f"COBYLA from scipy: {result.x}")
print("Best value found by COBYLA:", best_value)

# 调用调包方法进行优化
optimizer = ng.optimizers.Cobyla(parametrization=D, budget=budget)
recommendation = optimizer.minimize(objective)
best_solution = recommendation.value
best_value = objective(best_solution)

print(f"COBYLA from neverglad: {best_solution}")
print("Best value found by COBYLA:", best_value)

# 创建优化器（使用 CMA 算法）
optimizer = ng.optimizers.CMA(parametrization=D,budget=budget)  # 10维空间，预算200次迭代

# 执行优化
recommendation = optimizer.minimize(objective)

# 获取最优解
best_solution = recommendation.value
best_value = objective(best_solution)

print("Best solution found by CMA:", best_solution)
print("Best value found by CMA:", best_value)


COBYLA from scipy: [-6.80526468 -0.74184641]
Best value found by COBYLA: 64.67061669431543
COBYLA from neverglad: [ 1.15387748 -0.74189082]
Best value found by COBYLA: 0.9949590570932899
Best solution found by CMA: [0.15891469 0.25299632]
Best value found by CMA: 4.974791125313563
